In [1]:
import numpy as  np 
import matplotlib.pyplot as plt

In [2]:
from keras.datasets import mnist

Using TensorFlow backend.


In [3]:
(x_train,y_train),(x_test, y_test) = mnist.load_data()

In [4]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


<!-- SPLIT the MNIST dataset bs=cs we r goig to 
perform tranbsfer lerainng so 
we traned the model witn 0-4 digits and 
test tha mpodel with 5-9 didgit sets -->

In [5]:
pt_x_train = []
pt_y_train = []
pt_x_test = []
pt_y_test = []


tl_x_train = []
tl_y_train = []
tl_x_test = []
tl_y_test = []

In [6]:
# we deco=ided to eotk with only 20k images 

m = 20000
for i in range(m):
    if y_train[i]<5:
        pt_x_train.append(x_train[i]/255)
        pt_y_train.append(y_train[i])
    else:
        tl_x_train.append(x_train[i]/255)
        tl_y_train.append(y_train[i])


m2=10000  #
for i in range(m2):
    if y_test[i]<5:          #pre training y_train and x_train
        pt_x_test.append(x_test[i]/255)  #for noramlization
        pt_y_test.append(y_test[i] )
    else:
        tl_x_test.append(x_test[i]/255)
        tl_y_test.append(y_test[i] )
        

Reshapping the data 

In [7]:
pt_x_train = np.asarray(pt_x_train).reshape(-1,28,28,1)
pt_x_test = np.asarray(pt_x_test).reshape(-1,28,28,1)
tl_x_train = np.asarray(tl_x_train).reshape(-1,28,28,1)
tl_x_test = np.asarray(tl_x_test).reshape(-1,28,28,1)


In [8]:
from keras.utils import np_utils

In [9]:
pt_y_train = np_utils.to_categorical(np.asarray(pt_y_train))
pt_y_test = np_utils.to_categorical(np.asarray(pt_y_test))
tl_y_train = np_utils.to_categorical(np.asarray(tl_y_train))
tl_y_test = np_utils.to_categorical(np.asarray(tl_y_test))

In [10]:
print(pt_x_train.shape, pt_y_train.shape)
print(pt_x_test.shape, pt_y_test.shape)
print(tl_x_train.shape, tl_y_train.shape)
print(tl_x_test.shape, tl_y_test.shape)

(10225, 28, 28, 1) (10225, 5)
(5139, 28, 28, 1) (5139, 5)
(9775, 28, 28, 1) (9775, 10)
(4861, 28, 28, 1) (4861, 10)


Pretrend Model 

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Activation, MaxPool2D, Dropout

In [12]:
model = Sequential()

model.add(Conv2D(32,(5,5), input_shape=(28,28,1), activation='relu'))

model.add(Conv2D(16,(5,5), activation='relu'))

model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(8,(3,3), activation='relu'))

model.add(Flatten())

model.add(Dropout(0.4))

model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 16)        12816     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 8)           1160      
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)              

TRaining 

In [13]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])



In [15]:
import datetime

start  =  datetime.datetime.now()
model.fit(pt_x_train, pt_y_train,
         validation_data=(pt_x_test,pt_y_test),
         epochs=2,
         batch_size=16,
         verbose=2)
end = datetime.datetime.now()
print(end-start)

Train on 10225 samples, validate on 5139 samples
Epoch 1/2
 - 20s - loss: 0.0381 - accuracy: 0.9876 - val_loss: 0.0194 - val_accuracy: 0.9930
Epoch 2/2
 - 20s - loss: 0.0329 - accuracy: 0.9899 - val_loss: 0.0124 - val_accuracy: 0.9959
0:00:40.073923


In [16]:
model.layers

In [25]:
for layer in model.layers[:6]:
    layer.trainable=False

for layer in model.layers:
    print(layer.trainable)

False
False
False
False
False
False
True
True
True
True


first 6 --> False or frediging 
last 4 --> True ...

In [26]:
tl_model = Sequential(model.layers[:6])

tl_model.add(Dense(128))
tl_model.add(Activation('relu'))

tl_model.add(Dense(10))
tl_model.add(Activation('softmax'))

tl_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 16)        12816     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 8)           1160      
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)              

# results

In [27]:
tl_model.compile(loss="categorical_crossentropy",
                 optimizer="adam",
                 metrics=['accuracy']
                 )

In [28]:
start = datetime.datetime.now()
tl_model.fit(tl_x_train, tl_y_train,
            validation_data=(tl_x_test, tl_y_test),
            shuffle=True,
            batch_size=100,
            verbose=2)
end = datetime.datetime.now()
print(end-start)

Train on 9775 samples, validate on 4861 samples
Epoch 1/1
 - 4s - loss: 0.3542 - accuracy: 0.8929 - val_loss: 0.0922 - val_accuracy: 0.9722
0:00:04.739628
